In [46]:
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from transformers import AlbertTokenizer, TFAlbertForSequenceClassification
from transformers import XLNetTokenizer, TFXLNetForSequenceClassification
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [47]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
# tokenizer = AlbertTokenizer.from_pretrained("albert-base-v1")
tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
# tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

In [48]:
# example to training dataset
examples = [
    {"text": "I loved the movie", "label": 1},
    {"text": "I hated the movie", "label": 0}
]

In [49]:
# tokenization and data preparation
def encode_example(example):
    encoding = tokenizer.encode_plus(
        example['text'],
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        return_attention_mask=True,
        truncation=True,
        return_tensors='tf'
    )
    return {'input_ids': encoding['input_ids'][0], 'attention_mask': encoding['attention_mask'][0]}, example['label']

In [50]:
# convert examples to tensorFlow dataset
dataset = tf.data.Dataset.from_generator(
    lambda: (encode_example(ex) for ex in examples),
    output_signature=(
        {'input_ids': tf.TensorSpec(shape=(128,), dtype=tf.int32),
         'attention_mask': tf.TensorSpec(shape=(128,), dtype=tf.int32)},
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)

In [51]:
# batching
dataset = dataset.batch(2)

In [52]:
# loading the five models here
# model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
# model = TFRobertaForSequenceClassification.from_pretrained('roberta-base')
# model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
# model = TFAlbertForSequenceClassification.from_pretrained("albert-base-v1")
model = TFXLNetForSequenceClassification.from_pretrained("xlnet-base-cased")
# model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetForSequenceClassification: ['lm_loss']
- This IS expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForS

In [53]:
# training and params
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [54]:
# training
model.fit(dataset, epochs=100)

Epoch 1/100


1/1 [==============================] - 11s 11s/step - loss: 0.5616 - accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 1s 1s/step - loss: 0.4142 - accuracy: 1.0000
Epoch 3/100
1/1 [==============================] - 1s 841ms/step - loss: 0.5759 - accuracy: 1.0000
Epoch 4/100
1/1 [==============================] - 1s 1s/step - loss: 0.5547 - accuracy: 0.5000
Epoch 5/100
1/1 [==============================] - 1s 1s/step - loss: 0.4944 - accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 1s 1s/step - loss: 0.6842 - accuracy: 0.5000
Epoch 7/100
1/1 [==============================] - 1s 1s/step - loss: 0.6112 - accuracy: 0.5000
Epoch 8/100
1/1 [==============================] - 1s 1s/step - loss: 0.8210 - accuracy: 0.0000e+00
Epoch 9/100
1/1 [==============================] - 1s 1s/step - loss: 0.7833 - accuracy: 0.0000e+00
Epoch 10/100
1/1 [==============================] - 1s 1s/step - loss: 0.5291 - accuracy: 0.5000
Epoch 11/100
1/1 [====================

In [55]:
# testing data
test_examples = [
    "This is the best movie I have ever seen!",
    "Absolutely terrible! I will never watch this again.",
    "It was okay, not great but not bad either.",
    "you suck man",
    "this is so crazy and terrible to look at"
]

In [56]:
# preprocessing
test_encodings = tokenizer(test_examples, truncation=True, padding=True, max_length=128, return_tensors='tf')

In [57]:
# creating a tf dataset
test_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask']},
    tf.constant([0] * len(test_examples))
))
test_dataset = test_dataset.batch(3)

In [58]:
# prediction
predictions = model.predict(test_dataset)

2/2 [==============================] - 2s 59ms/step


In [59]:
# decoding predictions
predicted_labels = np.argmax(predictions.logits, axis=1)
sentiments = ["Positive" if label == 1 else "Negative" for label in predicted_labels]

In [60]:
# logging sentiment values
for sentence, sentiment in zip(test_examples, sentiments):
    print(f"sentence: \"{sentence}\" - sentiment: {sentiment}")

sentence: "This is the best movie I have ever seen!" - sentiment: Negative
sentence: "Absolutely terrible! I will never watch this again." - sentiment: Negative
sentence: "It was okay, not great but not bad either." - sentiment: Negative
sentence: "you suck man" - sentiment: Negative
sentence: "this is so crazy and terrible to look at" - sentiment: Negative
